In [1]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3693, done.
remote: Counting objects: 100% (3693/3693), done.
remote: Compressing objects: 100% (3085/3085), done.
remote: Total 3693 (delta 988), reused 1495 (delta 557), pack-reused 0
Receiving objects: 100% (3693/3693), 48.76 MiB | 20.69 MiB/s, done.
Resolving deltas: 100% (988/988), done.
Updating files: 100% (3355/3355), done.


In [2]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Reading package lists...
Building dependency tree...
Reading state information...
protobuf-compiler is already the newest version (3.6.1.3-2ubuntu5.2).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageColor, ImageFont, ImageDraw
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import time
from six import BytesIO

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

In [4]:
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
COCO17_HUMAN_POSE_KEYPOINTS = [(0, 1),
 (0, 2),
 (1, 3),
 (2, 4),
 (0, 5),
 (0, 6),
 (5, 7),
 (7, 9),
 (6, 8),
 (8, 10),
 (5, 6),
 (5, 11),
 (6, 12),
 (11, 12),
 (11, 13),
 (13, 15),
 (12, 14),
 (14, 16)]

In [5]:
def load_images():
  imgs = []
  for img_suffix in range(1, 12):
    img = Image.open('drive/MyDrive/TIES4911/task5/img-{}.jpg'.format(img_suffix))
    img = np.array(img, dtype=np.uint8)
    img = np.expand_dims(img, axis=0)
    imgs.append(img)

  return imgs

def load_image():
  image_data = tf.io.gfile.GFile('drive/MyDrive/TIES4911/task5/img-2.jpg', 'rb').read()
  image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)

In [6]:
def display_image(image):
  fig = plt.figure(figsize=(20, 32))
  plt.grid(False)
  plt.imshow(image)
  plt.show()

In [7]:
batch = load_images()

In [8]:
model_handle = 'https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1'
hub_model = hub.load(model_handle)

In [9]:
for img in batch:
  start_time = time.time()
  results = hub_model(img)
  end_time = time.time()
  result = {key:value.numpy() for key,value in results.items()}
  print("Found %d objects." % len(result["detection_scores"][0]))
  print("Inference time: ", end_time-start_time)

  label_id_offset = 0

  # Handle models with masks:
  image_np_with_mask = img.copy()

  if 'detection_masks' in result:
    # we need to convert np.arrays to tensors
    detection_masks = tf.convert_to_tensor(result['detection_masks'][0])
    detection_boxes = tf.convert_to_tensor(result['detection_boxes'][0])

    # Reframe the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              detection_masks, detection_boxes,
                img.shape[1], img.shape[2])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                        tf.uint8)
    result['detection_masks_reframed'] = detection_masks_reframed.numpy()

  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_mask[0],
        result['detection_boxes'][0],
        (result['detection_classes'][0] + label_id_offset).astype(int),
        result['detection_scores'][0],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=10,
        min_score_thresh=.10,
        agnostic_mode=False,
        instance_masks=result.get('detection_masks_reframed', None),
        line_thickness=8)

  display_image(image_np_with_mask[0])

Output hidden; open in https://colab.research.google.com to view.

In [12]:
print(len(result['detection_scores'][0]))

100
